In [1]:
from langchain_groq import ChatGroq
import os

In [2]:
import chromadb

In [3]:
GROG_API_KEY = os.environ.get("MY_GROG_API_KEY")

In [4]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    api_key=GROG_API_KEY,
    max_tokens=None,
    max_retries=2
)

In [5]:
client = chromadb.Client()
#collection = client.create_collection(name="cold_mail")
#collection = client.create_collection(name="cold_email")

In [ ]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Dhaka,Bangladesh"
    ],
    ids=["id1","id2"]
)

In [ ]:
all_docs = collection.get(ids=["id1"])
all_docs

In [22]:
results = collection.query(
    query_texts=['Query is about Bagels'],
    n_results=1
)
results

{'ids': [['id1']],
 'distances': [[1.7323635816574097]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [['This document is about New York']],
 'uris': None,
 'data': None}

In [29]:
collection.delete(ids=["id2"])
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

Capturing Website Data

In [14]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-43863")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer III, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer III, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. d

In [19]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """ 
    ### SRAPPED TEST FROM WEBSITE
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is extract the job postings and return them in JSON format containing following keys: 'role','experience','skills' and 'description'
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

In [20]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "Software Engineer III, ITC",
  "experience": "5-8 years",
  "skills": [
    "Marketing technology experience",
    "Adobe toolset experience (AEP, AJO, CJA)",
    "Python or Java",
    "Sql",
    "Data Engineering",
    "Airflow, EMR, Hive, Spark",
    "Microservices architecture",
    "Domain driven design",
    "RESTful API's",
    "CI/CD",
    "DevOps"
  ],
  "description": "We are looking for a Software engineer, to be a part of Nike’s Marketing Technology organization. This position will play a critical role, in developing Nike’s Customer Data platform, which supports and promotes our products and provides a unique user experience to all our customers in 166 countries globally."
}
```


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
jason_parser = JsonOutputParser()
jason_res = jason_parser.parse(res.content)
jason_res

JsonOutputParser()